In [7]:
import  pyodbc

import pandas as pd


### Connection to your SQL Database

In [18]:

# Connection parameters
server = input('Your server name')
database =('Your version of avdentureworks')

KeyboardInterrupt: Interrupted by user

In [19]:


# Establish connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')

# Create a cursor and fetch data from 'production.product'
cursor = conn.cursor()
cursor.execute("SELECT * FROM production.product")
rows = cursor.fetchall()

# Convert data to a DataFrame for 'production.product'
products = pd.DataFrame([tuple(row) for row in rows], columns=[column[0] for column in cursor.description])

# Create another cursor and fetch data from 'sales.salesorderdetail'
cursor.execute("SELECT * FROM sales.salesorderdetail")
rows = cursor.fetchall()

# Convert data to a DataFrame for 'sales.salesorderdetail'
sales_order_detail = pd.DataFrame([tuple(row) for row in rows], columns=[column[0] for column in cursor.description])

# Create another cursor and fetch data from 'VendorProduct'
cursor.execute("SELECT * FROM purchasing.ProductVendor")
rows = cursor.fetchall()
# Convert data to a DataFrame for 'purchasing.ProductVendor'
ProductVendor = pd.DataFrame([tuple(row) for row in rows], columns=[column[0] for column in cursor.description])
# Close the cursor and connection
cursor.close()
conn.close()




### Create the finished_goods table and the abc_analysis

In [21]:



# Filtering to include only finished goods
finished_goods = products[products['FinishedGoodsFlag'] == 1]

# Merging the Product and SalesOrderDetail tables on ProductID
sales_data = pd.merge(finished_goods, sales_order_detail, on='ProductID', how='left').fillna(0)

# Grouping the data by ProductID and Name and calculating sales totals
abc = sales_data.groupby(['ProductID', 'Name']).agg({'LineTotal': 'sum'}).reset_index()
abc['TotalSales'] = abc['LineTotal'].sum()
abc['RunningTotal'] = abc['LineTotal'].cumsum()
abc['RunningPct'] = (abc['RunningTotal'] / abc['TotalSales']) * 100

# Assigning ABC categories
def categorize(row):
    if row['RunningPct'] <= 80:
        return 'A'
    elif row['RunningPct'] > 80 and row['RunningPct'] <= 95:
        return 'B'
    else:
        return 'C'

abc['Category'] = abc.apply(categorize, axis=1)

# Display the final table
abc




C:\Users\anast\AppData\Local\Temp\ipykernel_25260\3822872448.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sales_data = pd.merge(finished_goods, sales_order_detail, on='ProductID', how='left').fillna(0)


,ProductID,Name,LineTotal,TotalSales,RunningTotal,RunningPct,Category
0,680,"HL Road Frame - Black, 58",0,109846381.399888,0,0E+6,A
1,706,"HL Road Frame - Red, 58",0,109846381.399888,0,0E+6,A
2,707,"Sport-100 Helmet, Red",157772.394392,109846381.399888,157772.394392,0.1436300334898067617523799319,A
3,708,"Sport-100 Helmet, Black",160869.517836,109846381.399888,318641.912228,0.2900795712769149891357551308,A
4,709,"Mountain Bike Socks, M",6060.388200,109846381.399888,324702.300428,0.2955967199738188811437448746,A
...,...,...,...,...,...,...,...
290,995,ML Bottom Bracket,0,109846381.399888,108422660.114582,98.70389787341010066774782197,C
291,996,HL Bottom Bracket,39581.442000,109846381.399888,108462241.556582,98.73993132439462291024847722,C
292,997,"Road-750 Black, 44",290298.624000,109846381.399888,108752540.180582,99.00420823575066325600784947,C
293,998,"Road-750 Black, 48",578174.312870,109846381.399888,109330714.493452,99.53055630976249366286993045,C
